In [1]:
import os
import numpy as np
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS,FK5
from IPython.display import clear_output
import astropy.units as u
from astropy.io import fits
import astropy

x1d_dir='/Users/codydirks/PGCC/x1d_files/'
tgas_dir='/Users/codydirks/PGCC/tgas_data/'
tgas_files=[x for x in os.listdir(tgas_dir) if x.endswith('.fits')]

# Performs a Simbad query for the given object,then 
# returns table entry of nearest Gaia source to the specified object's coordinates
def GaiaSearch(obj, radius=1*u.arcmin):
    # First, search for existing TYC/HIP identifiers, which we can use
    # to directly link to a Gaia source identifier
    idlist=Simbad.query_objectids(obj)
    tyc=[x[0] for x in idlist if x[0].startswith('TYC')]
    hip=[x[0] for x in idlist if x[0].startswith('HIP')]
    hip_id=-1
    tyc_id=''
    if len(hip)>0:
        hip_id=hip[0].split()[1]
        tgas_dat=Gaia.launch_job_async('SELECT source_id FROM gaiadr1.tgas_source WHERE hip='+hip_id,
                                       verbose=False).get_results()  
    elif len(tyc)>0:            
        tyc_id=tyc[0][4:]
        tgas_dat=Gaia.launch_job_async('SELECT source_id FROM gaiadr1.tgas_source WHERE tycho2_id=\''+tyc_id+'\'',
                                       verbose=False).get_results()
    
    # If either the TYC or HIP search returned a hit, extract the Gaia source id
    # and query for the full Gaia entry
    if len(tgas_dat)>0:
        gaia_source_id=tgas_dat[0]['source_id']
        gaia_dat=Gaia.launch_job_async('SELECT * FROM gaiadr1.gaia_source WHERE source_id='+str(gaia_source_id)
                            ,verbose=False).get_results()
        if len(gaia_dat)>0:
            return gaia_dat[0]
        else:
            return[]
    else: 
        # If no TYC/HIP identifier is found, instead search for the nearest Gaia source
        # using the Simbad coordinates of the object
        result=Simbad.query_object(obj)
        ra=result['RA'][0]
        dec=result['DEC'][0]
        coords=SkyCoord(ra+dec,unit=(u.hourangle,u.deg),frame='icrs',equinox='J2000')
        r = Gaia.cone_search_async(coordinate=coords, radius=1*u.arcmin,verbose=False).get_results()
        if len(r)>0:
            return r[0]
        else:
            return []
    
def get_gaia_plx(obj):
    row=GaiaSearch(obj)
    if len(row)>0 and type(row['parallax'])!=np.ma.core.MaskedConstant:
        return float(row['parallax']), float(row['parallax_error'])
    else:
        return np.nan,np.nan

Created TAP+ (v1.0) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
sightlines=[x for x in os.listdir(x1d_dir) if os.path.isdir(x1d_dir+x) and x.startswith('.')==False]

In [3]:
results=[]
for sl in sightlines:
    print sl
    results.append((sl, get_gaia_plx(sl)))
    clear_output()

In [8]:
filt=[x for x in results if np.isnan(x[1][0])==False]

In [12]:
for sl,(plx,plx_err) in filt:
    print sl,1000./plx

HD43582 1455.68232658
HD208266 865.029748437
HD27808 42.808617653
HD14818 3374.14530482
HD186688 656.981887466
HD116781 1261.72703616
HD38282 1684.56237835
HD21551 173.56849089
HD210809 4184.58611973
HD106943 375.891712797
HD138679 768.6135724
HD99857 1797.00283462
HD106343 1252.74112168
HD218915 2303.01353309
HD13745 2361.91038754
HD29647 166.973185617
HD94493 1521.05572321
HD203532 282.002056434
HD173297 1178.48507167
HD115071 1462.89033988
HD75309 995.934517694
HD52329 397.941520109
HD24534 724.581805707
HD29225 41.8059358897
HD115455 1214.40057117
HD101436 1024.07908901
HD72350 480.811133825
HD93205 1685.46868241
CPD-592603 9880.01346163
BD+631964 1561.75345208
HD124979 4358.56208635
HD1383 3300.06577585
HD31293 152.777452226
HD122879 1774.82221706
HD152590 1208.75391792
HD108927 302.119252545
HD37903 360.225286507
HD124314 -2245.91001171
HD40893 6577.55370814
HD72089 595.133108119
HD108002 3745.85573611
HD209339 735.409542145
HD208440 1414.19508135
HD44173 807.018889734
HD13841 29